In [85]:
import pandas as pd
import math 

In [2]:
lp_df = pd.read_csv("/home/eecs/wooders/experiments/stl/offline/lp_input.csv")
lp_df

,Unnamed: 0,key,slide_size,loss,n_fits
0,0,1,48,3.081649,21
1,1,1,12,3.398956,84
2,2,1,192,2.464865,6
3,3,1,6,3.470249,168
4,4,1,18,3.413212,56
...,...,...,...,...,...
995,995,100,24,1.467826,42
996,996,100,672,11.666722,2
997,997,100,18,1.399397,56
998,998,100,168,4.229301,6


In [32]:
lp_df[["slide_size", "n_fits"]].value_counts()

slide_size  n_fits
6           168       100
12          84        100
18          56        100
24          42        100
48          21        100
96          11        100
168         6         100
192         6         100
336         3         100
672         2         100
dtype: int64

In [47]:
slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}

In [6]:
lp_df[lp_df["slide_size"] == 96].sort_values(by="loss")

,Unnamed: 0,key,slide_size,loss,n_fits
153,153,16,96,0.045473,11
864,864,87,96,0.112649,11
816,816,82,96,0.266096,11
713,713,72,96,0.282656,11
331,331,34,96,0.305157,11
...,...,...,...,...,...
13,13,2,96,3.110132,11
495,495,50,96,3.405873,11
484,484,49,96,3.543840,11
114,114,12,96,4.943398,11


In [17]:
slides = lp_df.groupby("key")["slide_size"].apply(list).to_dict()
fits = lp_df.groupby("key")["n_fits"].apply(list).to_dict()
loss = lp_df.groupby("key")["loss"].apply(list).to_dict()

key_res = [] 
for key in slides.keys(): 
    s = slides[key]
    key_res.append({s[i]: loss[key][i] for i in range(len(s))})
  
key_res

[{48: 3.0816494824530696,
  12: 3.3989556789178463,
  192: 2.464865227267714,
  6: 3.4702486195433653,
  18: 3.413212252090997,
  96: 2.695120183845944,
  24: 3.277798947101666,
  336: 1.5228992594362534,
  168: 2.217797303350417,
  672: 2.008067817563694},
 {168: 8.966827486702613,
  672: 102.8675369132074,
  336: 34.30889555870277,
  96: 3.110132372402832,
  24: 0.5765546490010185,
  6: 0.4054272296348833,
  18: 0.4878143358160653,
  12: 0.4407618153179876,
  192: 11.031091264124887,
  48: 1.1202676429373517},
 {24: 1.00365239252522,
  96: 1.0141506405969605,
  672: 17.789155595023544,
  168: 1.5338837961301095,
  336: 4.950678387877023,
  192: 1.844273825193412,
  12: 1.0519596797597313,
  48: 0.9604535762276328,
  6: 1.082284088038825,
  18: 1.0326629200645534},
 {24: 0.337740780359018,
  96: 0.5484018834959654,
  672: 8.93955124924086,
  168: 1.0469228742705887,
  336: 3.2091020840213704,
  192: 1.2025053407875237,
  12: 0.3297445259923692,
  48: 0.3853437011439244,
  6: 0.3253818

In [19]:
baseline_loss = sum([res[96] for res in key_res])
baseline_cost = 11*100
print(baseline_cost, baseline_loss)

1100 143.8113846588952


In [69]:
total_cost = 0
total_loss = 0
thresh = 0.75

# slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}
options = [336, 192, 96, 48, 24]
for res in key_res: 
    slide = 336
    loss = res[slide]
    
    for op in options[1:]: 
        if res[op] < thresh * res[slide]: 
            slide = op 
        else: 
            break 
            
        
    print(slide, res[slide])
    total_cost += slide_to_fit[slide]
    total_loss += res[slide]
    
print(total_cost, total_loss)
        
        

336 1.5228992594362534
24 0.5765546490010185
96 1.0141506405969605
48 0.3853437011439244
96 0.7749053017537969
336 0.3320682038495794
192 1.134637024140515
336 1.83795417167952
336 1.7705759488768642
336 1.0052987785549428
96 1.8000288164502716
96 4.943397815125491
192 1.086854619732392
336 1.9745935453222452
336 0.7338521371265799
336 0.0424424529831269
96 0.5718397913735577
48 1.7181521743612689
96 0.8475316053536865
336 1.1981735902569526
24 0.474362524890145
96 0.7904547521383718
48 0.422581367829361
192 1.614528638578114
96 2.1286420154465096
336 2.716423572085956
192 2.721415129011734
192 1.3107098344229573
96 0.5751811731907418
96 0.7367844807152807
48 1.6645382329566214
96 1.4401145187821514
96 3.021150433966362
336 0.3021266803536334
96 2.4951063885707634
336 1.668222229286084
96 1.5178384884909062
336 2.2734555828207896
192 0.7104096549810902
336 1.239351681916019
336 1.0600566600056585
48 0.9110078275830592
336 1.486821163759272
336 1.3003244028136751
96 0.7331664079624011
3

In [71]:
baseline_loss = sum([res[48] for res in key_res])
baseline_cost = 21*100
print(baseline_cost, baseline_loss)

2100 119.8466294640843


In [137]:
total_cost = 0
total_loss = 0
thresh = 0.92

# slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}
#options = [336, 192, 96, 48, 24]
options = [192, 96, 48, 24]
for res in key_res: 
    slide = 336
    loss = res[slide]
    
    #print(res)
    for i in range(1, len(options), 1):
        print(options[i], slide, res[options[i]], math.pow(thresh, i), math.pow(thresh, i) * res[slide])
        if res[options[i]] < math.pow(thresh, i) * res[slide]: 
            slide = options[i]
            
        
    print(slide, res[slide])
    print()
    total_cost += slide_to_fit[slide]
    total_loss += res[slide]
    
print(total_cost, total_loss)
        

96 336 2.695120183845944 0.98 1.4924412742475284
48 336 3.0816494824530696 0.9603999999999999 1.4625924487625777
24 336 3.277798947101666 0.9411919999999999 1.433340599787326
336 1.5228992594362534

96 336 3.110132372402832 0.98 33.62271764752872
48 96 1.1202676429373517 0.9603999999999999 2.9869711304556796
24 48 0.5765546490010185 0.9411919999999999 1.0543869433914919
24 0.5765546490010185

96 336 1.0141506405969605 0.98 4.851664820119482
48 96 0.9604535762276328 0.9603999999999999 0.9739902752293208
24 48 1.00365239252522 0.9411919999999999 0.9039712223168381
48 0.9604535762276328

96 336 0.5484018834959654 0.98 3.144920042340943
48 96 0.3853437011439244 0.9603999999999999 0.5266851689095251
24 48 0.337740780359018 0.9411919999999999 0.3626824087670525
24 0.337740780359018

96 336 0.7749053017537969 0.98 2.8300752758294108
48 96 0.6977981422224551 0.9603999999999999 0.7442190518043464
24 48 0.6249791736336152 0.9411919999999999 0.6567620290746369
24 0.6249791736336152

96 336 0.3145

In [81]:
baseline_loss = sum([res[24] for res in key_res])
baseline_cost = 42*100
print(baseline_cost, baseline_loss)

4200 111.40812552020098


In [147]:
total_cost = 0
total_loss = 0
thresh = 0.99

# slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}
#options = [336, 192, 96, 48, 24]
options = [192, 96, 48, 24]
for res in key_res: 
    slide = 192
    loss = res[slide]
    
    #print(res)
    for i in range(1, len(options), 1):
        print(options[i], slide, res[options[i]], math.pow(thresh, i), math.pow(thresh, i) * res[slide])
        if res[options[i]] < math.pow(thresh, i) * res[slide]: 
            slide = options[i]
            
        
    total_cost += slide_to_fit[slide]
    total_loss += res[slide]
    
print(total_cost, total_loss)

96 192 2.695120183845944 0.99 2.440216574995037
48 192 3.0816494824530696 0.9801 2.4158144092450864
24 192 3.277798947101666 0.970299 2.3916562651526356
96 192 3.110132372402832 0.99 10.920780351483637
48 96 1.1202676429373517 0.9801 3.0482407381920154
24 48 0.5765546490010185 0.970299 1.0869945736744695
96 192 1.0141506405969605 0.99 1.825831086941478
48 96 0.9604535762276328 0.9801 0.9939690428490809
24 48 1.00365239252522 0.970299 0.931927144560096
96 192 0.5484018834959654 0.99 1.1904802873796485
48 96 0.3853437011439244 0.9801 0.5374886860143957
24 48 0.337740780359018 0.970299 0.37389860787624873
96 192 0.7749053017537969 0.99 1.2714182285724285
48 96 0.6977981422224551 0.9801 0.7594846862488963
24 48 0.6249791736336152 0.970299 0.677072839600306
96 192 0.3145606780934729 0.99 0.3279774321900165
48 96 0.3149514195431688 0.9801 0.3083009205994128
24 96 0.3136338349933766 0.970299 0.30521791139341864
96 192 0.9321275039504552 0.99 1.1232906538991099
48 96 0.8781617351225819 0.9801 

In [151]:
total_cost = 0
total_loss = 0
thresh = 0.9

# slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}
#options = [336, 192, 96, 48, 24]
options = [192, 96, 48, 24]
for res in key_res: 
    slide = 192
    loss = res[slide]
    
    #print(res)
    for i in range(1, len(options), 1):
        print(options[i], slide, res[options[i]], math.pow(thresh, i), math.pow(thresh, i) * res[slide])
        if res[options[i]] < math.pow(thresh, i) * res[slide]: 
            slide = options[i]
            
        
    total_cost += slide_to_fit[slide]
    total_loss += res[slide]
    
print(total_cost, total_loss)

96 192 2.695120183845944 0.9 2.218378704540943
48 192 3.0816494824530696 0.81 1.9965408340868485
24 192 3.277798947101666 0.7290000000000001 1.7968867506781638
96 192 3.110132372402832 0.9 9.927982137712398
48 96 1.1202676429373517 0.81 2.519207221646294
24 48 0.5765546490010185 0.7290000000000001 0.8166751117013294
96 192 1.0141506405969605 0.9 1.6598464426740709
48 96 0.9604535762276328 0.81 0.8214620188835381
24 96 1.00365239252522 0.7290000000000001 0.7393158169951842
96 192 0.5484018834959654 0.9 1.0822548067087714
48 96 0.3853437011439244 0.81 0.444205525631732
24 48 0.337740780359018 0.7290000000000001 0.28091555813392094
96 192 0.7749053017537969 0.9 1.1558347532476623
48 96 0.6977981422224551 0.81 0.6276732944205755
24 96 0.6249791736336152 0.7290000000000001 0.564905964978518
96 192 0.3145606780934729 0.9 0.2981613019909241
48 192 0.3149514195431688 0.81 0.2683451717918317
24 192 0.3136338349933766 0.7290000000000001 0.24151065461264853
96 192 0.9321275039504552 0.9 1.0211733

In [169]:
total_cost = 0
total_loss = 0
thresh = 0.99

# slide_to_fit = {96: 11, 192: 6, 336: 3, 48: 21, 24: 42}
#options = [336, 192, 96, 48, 24]
options = [192, 96, 48, 24]
for res in key_res: 
    slide = 192
    loss = res[slide]
    
    #print(res)
    for i in range(1, len(options), 1):
        print(options[i], slide, res[options[i]], math.pow(thresh, i), math.pow(thresh, i) * res[slide])
        if res[options[i]] < math.pow(thresh, i) * res[slide]: 
            slide = options[i]
            
        
    total_cost += slide_to_fit[slide]
    total_loss += res[slide]
    
print(total_cost, total_loss)

96 192 2.695120183845944 0.99 2.440216574995037
48 192 3.0816494824530696 0.9801 2.4158144092450864
24 192 3.277798947101666 0.970299 2.3916562651526356
96 192 3.110132372402832 0.99 10.920780351483637
48 96 1.1202676429373517 0.9801 3.0482407381920154
24 48 0.5765546490010185 0.970299 1.0869945736744695
96 192 1.0141506405969605 0.99 1.825831086941478
48 96 0.9604535762276328 0.9801 0.9939690428490809
24 48 1.00365239252522 0.970299 0.931927144560096
96 192 0.5484018834959654 0.99 1.1904802873796485
48 96 0.3853437011439244 0.9801 0.5374886860143957
24 48 0.337740780359018 0.970299 0.37389860787624873
96 192 0.7749053017537969 0.99 1.2714182285724285
48 96 0.6977981422224551 0.9801 0.7594846862488963
24 48 0.6249791736336152 0.970299 0.677072839600306
96 192 0.3145606780934729 0.99 0.3279774321900165
48 96 0.3149514195431688 0.9801 0.3083009205994128
24 96 0.3136338349933766 0.970299 0.30521791139341864
96 192 0.9321275039504552 0.99 1.1232906538991099
48 96 0.8781617351225819 0.9801 